In [1]:
import pandas as pd
from core.config import GENERATED_DATA_FOLDER
from core.utils.file_path import solve_path

In [2]:
delta_file = solve_path('delta_area_construida.csv', 
                        GENERATED_DATA_FOLDER)

delta = pd.read_csv(delta_file, sep=';',
                   dtype={'quadra' : 'object'})

In [3]:
delta.head()

,quadra,area_construida_vertical_2013,area_construida_vertical_2023,delta_pde
0,001003,6832.0,6832.0,0.0
1,001004,3796.0,3796.0,0.0
2,001005,3594.0,3594.0,0.0
3,001006,2517.0,2517.0,0.0
4,001007,4827.0,8350.0,3523.0


In [4]:
delta.quadra.duplicated().any()

False

In [5]:
len(delta)

10245

In [6]:
dist_quadras_file = solve_path('quadras_com_distritos.csv',
                         GENERATED_DATA_FOLDER)
dist_quadras = pd.read_csv(dist_quadras_file, sep=';',
                           dtype={'quadra' : 'object'})

In [7]:
dist_quadras.head()

,quadra,qd_id_orig,qd_setor,qd_fiscal,qd_tipo,qd_subqua,qd_area,qd_situac,qd_id,qd_tx_tipo,ds_areamt,ds_subpref,ds_sigla,ds_nome,ds_areakm,ds_cd_sub,ds_codigo
0,001001,8893734,1,1,M,1,6463.700424,1,172817,AREA PUBLICA MUNICIPAL,2396993.441,SE,REP,REPUBLICA,2.397,9.0,66.0
1,001002,8893731,1,2,M,1,3848.083386,1,142629,AREA PUBLICA MUNICIPAL,2193819.874,SE,SEE,SE,2.194,9.0,78.0
2,001003,8893737,1,3,F,1,21744.900269,1,180769,FISCAL,4205896.639,SE,BRE,BOM RETIRO,4.206,9.0,9.0
3,001004,8893678,1,4,F,1,14803.411266,1,191772,FISCAL,4205896.639,SE,BRE,BOM RETIRO,4.206,9.0,9.0
4,001005,8893742,1,5,F,1,9177.188410,1,192049,FISCAL,4205896.639,SE,BRE,BOM RETIRO,4.206,9.0,9.0


In [8]:
len(dist_quadras)

47424

In [9]:
dist_quadras.quadra.duplicated().any()

False

In [10]:
merged = pd.merge(delta, dist_quadras, on='quadra', how='outer')

In [11]:
merged['delta_pde'].isnull().sum()

37183

In [12]:
merged['ds_nome'].isnull().sum()

28

In [13]:
merged['ds_nome'].isnull().sum()/len(merged)

0.0005903685586573333

In [14]:
merged = merged[merged['ds_nome'].notnull()].copy()

In [15]:
merged.keys()

Index(['quadra', 'area_construida_vertical_2013',
       'area_construida_vertical_2023', 'delta_pde', 'qd_id_orig', 'qd_setor',
       'qd_fiscal', 'qd_tipo', 'qd_subqua', 'qd_area', 'qd_situac', 'qd_id',
       'qd_tx_tipo', 'ds_areamt', 'ds_subpref', 'ds_sigla', 'ds_nome',
       'ds_areakm', 'ds_cd_sub', 'ds_codigo'],
      dtype='object')

In [16]:
areas = [
        'area_construida_vertical_2013',
        'area_construida_vertical_2023', 
        'delta_pde'
    ]

In [17]:
for col in areas:
    merged[col] = merged[col].fillna(0)

In [18]:
df_final = merged[areas + ['ds_nome']]

In [19]:
delta_por_distrito = df_final.groupby('ds_nome').sum()

In [21]:
delta_por_distrito = delta_por_distrito.reset_index()

In [22]:
delta_por_distrito.head()

,ds_nome,area_construida_vertical_2013,area_construida_vertical_2023,delta_pde
0,AGUA RASA,1622759.0,2108421.0,485662.0
1,ALTO DE PINHEIROS,2065880.0,2243481.0,177601.0
2,ANHANGUERA,0.0,1377.0,1377.0
3,ARICANDUVA,322938.0,640265.0,317327.0
4,ARTUR ALVIM,581546.0,657230.0,75684.0


In [23]:
fname = solve_path('delta_area_construida_distrito.csv',
                  GENERATED_DATA_FOLDER)
delta_por_distrito.to_csv(fname, index=False, sep=';')